# Create noleap calendar version of COSMO-REA6 forcing data

created 2023-04-14

by Eva Lieungh, Lasse T. Keetz, Hui Tang, Yeliz Yilmaz. 

COMSO reanalysis data uses a gregorian calendar with leap years. Running long simulations (necessary for spin-up) with COMSO reanalysis data proved problematic, because the leap years don't align and the model terminates prematurely after ~100 years. 

This notebook will...
1. read in the cosmo reanalysis atmospheric forcing files
2. change the calendar attribute for all files to noleap
3. remove leap days (Feb 29) from all files
3. save the modified data

In [19]:
# import libraries
import zipfile # for unzipping
import shutil # easiest whole-directory zipping
from pathlib import Path  # For easy path handling

Download COSMOREA data from evalieungh/FATES_INCLINE repo if necessary:

In [ ]:
%%bash
pwd
cd ../data
pwd
wget https://raw.githubusercontent.com/evalieungh/FATES_INCLINE/main/data/ALP4_cosmorea.zip

In [20]:
# set path to data, where we have the original (gregorian) data and will save the modified version
cosmo_path = str(Path(f"C:/Users/evaler/OneDrive - Universitetet i Oslo/Eva/PHD/FATES_INCLINE/data"))

Unzip folder:

In [ ]:
print("extracting ", cosmo_path + "/ALP4_cosmorea.zip")
with zipfile.ZipFile(cosmo_path + "/ALP4_cosmorea.zip", 'r') as zip_ref:
    zip_ref.extractall(cosmo_path + "/ALP4_cosmorea")

******************************

### Remove leap years

I.e., remove all variable values for Feb 29 all years. In a loop through all the files in the datm folder, if the file ends with -02 (February), remove all data values when day = 29. NB! This overwrites existing files, so make sure the originals are stored safely somewhere else.


In [13]:
%%bash
pwd
cd ../data/ALP4_cosmorea_noleap/datmdata
for f in clm1pt_ALP4*-02.nc; do cp $f ori_$f ; cdo delete,month=2,day=29 ori_$f $f; rm ori_$f; done

/mnt/c/Users/evaler/OneDrive - Universitetet i Oslo/Eva/PHD/FATES_INCLINE/src


cdo    delete (Warning): Day >29< not found!


cdo    delete: Processed 2694 values from 18 variables over 224 timesteps [0.05s 46MB].
cdo    delete: Processed 2694 values from 18 variables over 232 timesteps [0.04s 47MB].


cdo    delete (Warning): Day >29< not found!


cdo    delete: Processed 2694 values from 18 variables over 224 timesteps [0.04s 47MB].


...

cdo    delete: Processed 2694 values from 18 variables over 224 timesteps [0.05s 47MB].
cdo    delete: Processed 2694 values from 18 variables over 232 timesteps [0.04s 47MB].


cdo    delete (Warning): Day >29< not found!


cdo    delete: Processed 2694 values from 18 variables over 224 timesteps [0.04s 47MB].


cdo    delete (Warning): Day >29< not found!


cdo    delete: Processed 2694 values from 18 variables over 224 timesteps [0.04s 47MB].


Check that it worked:

In [15]:
%%bash
cd ../data/ALP4_cosmorea_noleap/datmdata
ncdump -h clm1pt_ALP4_2000-02.nc

netcdf clm1pt_ALP4_2000-02 {
dimensions:
	time = UNLIMITED ; // (224 currently)
	scalar = 1 ;
	lon = 1 ;
	lat = 1 ;
variables:
	double time(time) ;
		time:standard_name = "time" ;
		time:units = "hours since 2000-2-1 01:00:00" ;
		time:calendar = "standard" ;
		time:axis = "T" ;
	float EDGEW(scalar) ;
		EDGEW:long_name = "western edge in atmospheric data" ;
		EDGEW:units = "degrees_east" ;
		EDGEW:mode = "time-invariant" ;
	float EDGEE(scalar) ;
		EDGEE:long_name = "eastern edge in atmospheric data" ;
		EDGEE:units = "degrees_east" ;
		EDGEE:mode = "time-invariant" ;
	float EDGES(scalar) ;
		EDGES:long_name = "southern edge in atmospheric data" ;
		EDGES:units = "degrees_north" ;
		EDGES:mode = "time-invariant" ;
	float EDGEN(scalar) ;
		EDGEN:long_name = "northern edge in atmospheric data" ;
		EDGEN:units = "degrees_north" ;
		EDGEN:mode = "time-invariant" ;
	float LONGXY(lat, lon) ;
		LONGXY:long_name = "longitude" ;
		LONGXY:units = "degrees_east" ;
		LONGXY:mode = "time-invariant" 

***********************

## Change the value of the calendar attribute from 'standard' to 'noleap'

Calendar is specified within the time variable. The most reliable way to change it is with 'ncatted' from the nco library.

In [16]:
%%bash
pwd
cd ../data/ALP4_cosmorea_noleap/datmdata
for f in clm1pt_ALP4*; do ncatted -O -a calendar,time,o,c,noleap $f; done

/mnt/c/Users/evaler/OneDrive - Universitetet i Oslo/Eva/PHD/FATES_INCLINE/src


In [17]:
%%bash
cd ../data/ALP4_cosmorea_noleap/datmdata
ncdump -h clm1pt_ALP4_2000-02.nc

netcdf clm1pt_ALP4_2000-02 {
dimensions:
	time = UNLIMITED ; // (224 currently)
	scalar = 1 ;
	lon = 1 ;
	lat = 1 ;
variables:
	double time(time) ;
		time:standard_name = "time" ;
		time:units = "hours since 2000-2-1 01:00:00" ;
		time:calendar = "noleap" ;
		time:axis = "T" ;
	float EDGEW(scalar) ;
		EDGEW:long_name = "western edge in atmospheric data" ;
		EDGEW:units = "degrees_east" ;
		EDGEW:mode = "time-invariant" ;
	float EDGEE(scalar) ;
		EDGEE:long_name = "eastern edge in atmospheric data" ;
		EDGEE:units = "degrees_east" ;
		EDGEE:mode = "time-invariant" ;
	float EDGES(scalar) ;
		EDGES:long_name = "southern edge in atmospheric data" ;
		EDGES:units = "degrees_north" ;
		EDGES:mode = "time-invariant" ;
	float EDGEN(scalar) ;
		EDGEN:long_name = "northern edge in atmospheric data" ;
		EDGEN:units = "degrees_north" ;
		EDGEN:mode = "time-invariant" ;
	float LONGXY(lat, lon) ;
		LONGXY:long_name = "longitude" ;
		LONGXY:units = "degrees_east" ;
		LONGXY:mode = "time-invariant" ;


Re-zip folder

In [23]:
shutil.make_archive(cosmo_path + "/ALP4_cosmorea_noleap",
                    'zip', cosmo_path + "/ALP4_cosmorea_noleap")
print("zipping complete")

zipping complete


Finally, commit and push changes back to github repository so the data can be downloaded from there.